In [54]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import datetime
import sqlite3
import sys
import traceback
import numpy as np
import Data.database_handler as dbHandler
from torchvision import transforms, utils
import datetime as dt
sys.path.append('..')
#%run Map_grid/map.ipynb import CalculateGrid

#Connecting to the SQLite database
dataAmount = 200000
dbPath = r'Data\datasetNY.db'
gridSize = 5
chunkAmount = 1000
chunkSize = dataAmount / chunkAmount

class AccidentDataset(Dataset):
    def __init__(self, db_path, transform=None):
        self.coordinates = dbHandler.get_n_data_datetime_converted(db_path, dataAmount)
        self.coordinates = pd.DataFrame(self.coordinates, columns=['datetime', 'latitude', 'longitude'])
        
        #split into 500 chunks using numpy
        self.coordinates = np.array_split(self.coordinates, chunkAmount)

        #process each chunk and merge it back into one dataframe
        self.grids = []
        grid_lower_lat, grid_lower_long = 40.54, -74.15
        grid_upper_lat, grid_upper_long = 40.91, -73.70
        grid_lat_step = (grid_upper_lat - grid_lower_lat) / gridSize
        grid_long_step = (grid_upper_long - grid_lower_long) / gridSize
        for i in range(len(self.coordinates)-1):
            grid = np.zeros((gridSize, gridSize))
            for index, row in self.coordinates[i].iterrows():
                coordinates = row['latitude'], row['longitude']
                for j in range(gridSize):
                    for k in range(gridSize):
                        lat_lower = grid_lower_lat + j * grid_lat_step
                        lat_upper = grid_lower_lat + (j + 1) * grid_lat_step
                        long_lower = grid_lower_long + k * grid_long_step
                        long_upper = grid_lower_long + (k + 1) * grid_long_step
                        if lat_lower <= float(coordinates[0]) < lat_upper and long_lower <= float(coordinates[1]) < long_upper:
                            grid[j][k] += 1
                            break
            self.grids.append(grid/chunkSize)
        self.grids = np.array(self.grids)
        self.transform = transform      

    def __len__(self):
        return len(self.grids)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        grid = self.grids[idx]
        grid = torch.from_numpy(grid).float()

        max_index = np.argmax(grid)
        max_index = np.array(max_index)
        #get indicies of the highest value and nearest neighbours that are within 10% of the highest value
        #max_index = np.unravel_index(max_index, grid.shape)
        #max_index = np.array(max_index)
        #max_index = max_index.flatten()
        Jonasu_wa_kanji_o_motte_imasu = max_index.item() // gridSize, max_index.item() % gridSize
        max_value = grid[Jonasu_wa_kanji_o_motte_imasu[0]][Jonasu_wa_kanji_o_motte_imasu[1]].item()
        indicies = np.empty((0,2), float)
        for i in range(gridSize):
            for j in range(gridSize):
                if grid[i][j].item() >= max_value * 0.9:
                    indicies = np.append(indicies, i * gridSize + j)

        indicies = torch.from_numpy(indicies).reshape(-1, 1).flatten()
        indicies = torch.nn.functional.pad(indicies, (0, gridSize - len(indicies)), value=-1)
        return grid, indicies

accident_dataset = AccidentDataset(dbPath)

#Create new array with 60% of the data
train_size = int(0.6 * len(accident_dataset))
test_size = len(accident_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(accident_dataset, [train_size, test_size])

print(len(train_dataset))
print(len(test_dataset))
print(len(accident_dataset))

print('あいうえお') #aiueo
print('らりるれろ') #rarirurero
print('かきくけこ') #kakikukeko
print('さしすせそ') #sashisuseso
print('たちつてと') #tachitsuteto
print('なにぬねの') #naninuneno
print('はひふへほ') #hahifuheho
print('まみむめも') #mamimumemo
print('やゆよ') #yayuyo
print('わをん') #wawon
print('がぎぐげご') #gagigugego
print('ざじずぜぞ') #zazizuzezo
print('だぢづでど') #dazidizudezo
print('ばびぶべぼ') #babibubebo
print('ぱぴぷぺぽ') #papipupepo
print('っ') #small tsu

#now in katakana
print('アイウエオ') #aiueo
print('ラリルレロ') #rarirurero
print('カキクケコ') #kakikukeko
print('サシスセソ') #sashisuseso
print('タチツテト') #tachitsuteto
print('ナニヌネノ') #naninuneno
print('ハヒフヘホ') #hahifuheho
print('マミムメモ') #mamimumemo
print('ヤユヨ') #yayuyo
print('ワヲン') #wawon
print('ガギグゲゴ') #gagigugego
print('ザジズゼゾ') #zazizuzezo
print('ダヂヅデド') #dazidizudezo
print('バビブベボ') #babibubebo
print('パピプペポ') #papipupepo
print('ッ') #small tsu

#now in kanji
print('日本語') #nihongo
print('漢字') #kanji
print('ひらがな') #hiragana
print('カタカナ') #katakana
print('アルファベット') #alphabet
print('数字') #number
print('記号') #symbol
print('英語') #english
print('中国語') #chinese
print('韓国語') #korean

#Now in your mom
print('MILF') #Jonas
print('BOOBA') #NotJonas
print('YEEEHHHAAAWWW') #Jonasu

print('To fly is to controllably float using air as leverage, despite the gravitational pull of some giant mass') #JonasDescription

#Create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=gridSize*gridSize, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=gridSize*gridSize, shuffle=False)

# define the class for multilinear regression
class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(gridSize*gridSize, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, gridSize),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


# define the class for multilinear regression
# building the model object
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Using {device} device')

model = Network().to(device)
if os.path.exists("model.pth"):
    model.load_state_dict(torch.load("model.pth"))
    print("Loaded model from model.pth")
else:
    print("No model found, creating new model")

# define the loss function
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# define the training loop
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    print(size)
    model.train()
    print("Training model")
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        #print(X)
        #print(y)
        pred = model(X)
        #print('pred: ', pred)
        #print('y: ', y)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    print("Finished training model")

def test(dataloader, model, loss_fn):
    print("Testing model")
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

[1517472000.0, '40.886127', '-73.81532']
599
400
999
あいうえお
らりるれろ
かきくけこ
さしすせそ
たちつてと
なにぬねの
はひふへほ
まみむめも
やゆよ
わをん
がぎぐげご
ざじずぜぞ
だぢづでど
ばびぶべぼ
ぱぴぷぺぽ
っ
アイウエオ
ラリルレロ
カキクケコ
サシスセソ
タチツテト
ナニヌネノ
ハヒフヘホ
マミムメモ
ヤユヨ
ワヲン
ガギグゲゴ
ザジズゼゾ
ダヂヅデド
バビブベボ
パピプペポ
ッ
日本語
漢字
ひらがな
カタカナ
アルファベット
数字
記号
英語
中国語
韓国語
MILF
BOOBA
YEEEHHHAAAWWW
Using cpu device
No model found, creating new model
Epoch 1
-------------------------------
599
Training model
loss: 30.809083  [   25/  599]
Finished training model
Testing model


RuntimeError: The size of tensor a (25) must match the size of tensor b (5) at non-singleton dimension 1